In [41]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Dropout, GRU, Conv1D, MaxPooling1D, Input, concatenate
from keras.datasets import imdb
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

In [2]:
# read in features of one store (TX_2)
features = pd.read_pickle('TX_2_features.pkl').dropna()

In [7]:
from keras import backend as K

def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [19]:
NUM_LAG_DAYS= 28

train = features[(features.d<=1863) & (features.d>=1069+NUM_LAG_DAYS)].dropna()
val = features[(features.d > 1863) & (features.d < 1914)].dropna()
test = features[features.d >= 1914]
drop_cols = ['sales','id','d','wm_yr_wk','original_id','store_id','item_id']
num_weeks = NUM_LAG_DAYS // 7
for j in range(1,num_weeks+1):
    drop_cols += ['lag'+str(j) for j in range(j*7-6,j*7)]
drop_cols += ['event_name_1','event_name_2','event_type_1','event_type_2']
x_train = train.drop(drop_cols,axis=1)
y_train = train.sales.values
x_val = val.drop(drop_cols,axis=1)
y_val = val.sales.values    

In [6]:
x_train.shape

(2273370, 48)

In [14]:
# training configurations
num_epochs = 100
batch_size = 128
num_nodes = 128
num_layers = 3
dropout = 0.2
loss_fn = rmse
optimizer = 'adam'

In [15]:
mlp_model = Sequential()
mlp_model.add(Dense(num_nodes, input_dim=x_train.shape[1], activation='relu'))
mlp_model.add(Dropout(dropout))
for i in range(num_layers-1):
    mlp_model.add(Dense(num_nodes, activation='relu'))
    mlp_model.add(Dropout(dropout))
mlp_model.add(Dense(1, activation='relu'))
mlp_model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
# mc = ModelCheckpoint('tfidf_best_model.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
mlp_model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=num_epochs,
                validation_data=(x_val, y_val),
                callbacks=[es],
                verbose=1)

Train on 2273370 samples, validate on 152450 samples
Epoch 1/100
2273370/2273370 [==============================] - 48s 21us/step - loss: 2.0013 - accuracy: 0.5027 - val_loss: 1.9624 - val_accuracy: 0.4326
Epoch 2/100
2273370/2273370 [==============================] - 46s 20us/step - loss: 1.9304 - accuracy: 0.5022 - val_loss: 1.8894 - val_accuracy: 0.4774
Epoch 3/100
2273370/2273370 [==============================] - 43s 19us/step - loss: 1.9251 - accuracy: 0.5034 - val_loss: 1.6508 - val_accuracy: 0.4174
Epoch 4/100
2273370/2273370 [==============================] - 43s 19us/step - loss: 1.9182 - accuracy: 0.5039 - val_loss: 1.7145 - val_accuracy: 0.3875
Epoch 5/100
2273370/2273370 [==============================] - 43s 19us/step - loss: 1.9186 - accuracy: 0.5053 - val_loss: 1.7206 - val_accuracy: 0.3937
Epoch 6/100
2273370/2273370 [==============================] - 45s 20us/step - loss: 1.9138 - accuracy: 0.5056 - val_loss: 1.6840 - val_accuracy: 0.3987
Epoch 7/100
2273370/2273370 [

In [59]:
recurrent_dropout=0.2

In [63]:
lstm_model = Sequential()
lstm_model.add(Embedding(x_train.shape[1], num_nodes))
lstm_model.add(LSTM(num_nodes, dropout=dropout, recurrent_dropout=recurrent_dropout,return_sequences=True, input_shape=(batch_size, x_train.shape[1])))
lstm_model.add(LSTM(num_nodes, dropout=dropout, recurrent_dropout=recurrent_dropout,return_sequences=True, input_shape=(batch_size, x_train.shape[1])))
lstm_model.add(LSTM(num_nodes, dropout=dropout, recurrent_dropout=recurrent_dropout))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(loss=loss_fn,optimizer=optimizer,metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
# mc = ModelCheckpoint('tfidf_best_model.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
lstm_model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=num_epochs,
                validation_data=(x_val, y_val),
                callbacks=[es],
                verbose=1)

ValueError: Input 0 is incompatible with layer lstm_16: expected ndim=3, found ndim=4

In [52]:
# split features
drop_cols = ['id','d','wm_yr_wk','original_id','store_id','item_id']
drop_cols += ['event_name_1','event_name_2','event_type_1','event_type_2']
mv_features = features.drop(drop_cols,axis=1)
lag_cols = ['lag'+str(i) for i in range(1,29)]
other_cols = [col for col in mv_features.columns if col not in lag_cols+['sales']]
lag_features = mv_features[lag_cols]
other_features = mv_features[other_cols]
labels = mv_features['sales']

In [53]:
print(lag_features.shape)
print(other_features.shape)

(2425820, 28)
(2425820, 40)


In [54]:
cut = lag_features.shape[0]//5
lag_features_train = lag_features[cut:]
other_features_train = other_features[cut:]
labels_train = labels[cut:]
lag_features_val = lag_features[:cut]
other_features_val = other_features[:cut]
labels_val = labels[:cut]

In [56]:
lags = Input(shape=(28,1),name='lags')
others = Input(shape=(40,1),name='others')

lagLayers = LSTM(64)(lags)
otherLayers = LSTM(64)(others)

output = concatenate(
    [
        lagLayers,
        otherLayers
    ]
)
output = Dense(56,activation='relu',name='combined')(output)

mv_model = Model(
    inputs = [lags,others],
    outputs = [output]
)
mv_model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])

ValueError: Input 0 is incompatible with layer lstm_9: expected ndim=3, found ndim=2

In [55]:
# mv_model.fit(
#     [lag_features_train,other_features_train],[labels_train],
#     validation_data=([lag_features_val,other_features_val],[labels_val]),
#     epochs = 1,
#     batch_size=128,
# )

ValueError: Error when checking input: expected lags to have 3 dimensions, but got array with shape (1940656, 28)